In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Open the Corn file.
corn = pd.read_csv('corn.csv')
corn.head()

In [ ]:
# Pull out the insect resistant variety and the herbicide-tolerant vareity.  The insect
# resistent consists of the rows 3-16 and columns 0-18.
corn_insect = corn.iloc[3:17,0:19]

# Next we want to assing the years to be the column names.  Oddly some of them show up as floats
# but that will be fixed later.
corn_insect.columns = corn.iloc[1,0:19]
corn_insect = corn_insect.reset_index(drop=True)
corn_insect.head()

In [ ]:
# The herbiside tolorenat consists of the rows 3-16 and columns 1 (for state) and 19-37 (for years)
s = [0]
s.extend(list(range(19,37)))
corn_herbicide = corn.iloc[3:17,s]
corn_herbicide.head()

# Assign the years as column headers
corn_herbicide.columns = corn.iloc[1,s]
corn_herbicide = corn_herbicide.reset_index(drop=True)
corn_herbicide.head()

In [ ]:
# Melt the insert resistant data into a Tidy data frame, and add in the missing columns.
corn_insect_tidy = pd.melt(corn_insect, id_vars=['State'], var_name='Year', value_name='Value')
corn_insect_tidy['Crop'] = 'corn'
corn_insect_tidy['Crop Title'] = 'Genetically engineered (GE) corn'
corn_insect_tidy['Variety'] = 'Insect-resistant (Bt) only'
corn_insect_tidy['Unit'] = 'Percent of all corn planted'
corn_insect_tidy.head()

In [ ]:
# Melt the herbicide data into a Tidy data frame, and add in the missing column
corn_herbicide_tidy = pd.melt(corn_herbicide, id_vars=['State'], var_name='Year', value_name='Value')
corn_herbicide_tidy['Crop'] = 'corn'
corn_herbicide_tidy['Crop Title'] = 'Genetically engineered (GE) corn'
corn_herbicide_tidy['Variety'] = 'Herbicide-tolerant only'
corn_herbicide_tidy['Unit'] = 'Percent of all corn planted'
corn_herbicide_tidy.head()

In [ ]:
# Merge the insect resistant and herbicide tolerant tidy'ed data frames
corn_tidy = pd.concat([corn_insect_tidy, corn_herbicide_tidy])
corn_tidy.head()

In [ ]:
# Let's make sure our Year and Value are integers and not stings, but we must drop NAs first
corn_tidy = corn_tidy.dropna()
corn_tidy['Year'] = corn_tidy['Year'].astype('int')
corn_tidy['Value'] = corn_tidy['Value'].astype('int')
corn_tidy.head()

In [ ]:
# Plot the percentage of GE corn produced in Iowa.
State = 'Iowa'
Variety = 'Insect-resistant (Bt) only'
xticks = corn_tidy['Year'].drop_duplicates().sort_values()
corn_tidy[(corn_tidy['State'] == State) & (corn_tidy['Variety'] == Variety)].plot(
    kind='line', x='Year', y="Value", 
    xticks=xticks, rot=45,
    title= 'Percentage of Insect-resistant (Bt) only GE corn in Iowa'
)

In [ ]:
# According to the Pandas visualization documentation (https://pandas.pydata.org/pandas-docs/stable/visualization.html),
# to do a stacked box plot with year as the x-axis and the states values as stacked we have to rearrange
# our data frame so that our states are the column headers and the years are the rows. We can do this with
# the pivot command.
cit = corn_tidy[corn_tidy['Variety'] == "Insect-resistant (Bt) only"]
citp = cit[['Year', 'State', 'Value']].pivot(index='Year', columns='State')
citp.head()

In [ ]:
# Now we can do a boxplot. Unfortunately, the legend sits rith over top of the
# plot, so we hide it.  We can use matplotlib to create a plot without it.
citp.plot(kind='bar', rot=45, legend=False, figsize=(15, 4), 
          title='Percentage of Insect-resistant (Bt) only GE corn in all states.')